In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
x = np.load('/content/drive/MyDrive/data.T/X.npy')
y = np.load('/content/drive/MyDrive/data.T/Y.npy')

In [ ]:
d = np.genfromtxt('/content/drive/MyDrive/data.T/Vert.csv', delimiter = ',')
d = d[1:]

In [ ]:
y.shape

(15002, 4, 15002)

In [ ]:
dip_y = np.zeros((15002,4,3))
for i in range(15002):
  for j in range(3):
    dip_y[i,j] = d[y[i][j].nonzero()[0][0]]

In [ ]:
for i in range(15002):
    scaler = StandardScaler().fit(x[i])
    x[i] = scaler.transform(x[i])

In [ ]:
test_x = np.load('/content/drive/MyDrive/data.T/testX.npy')
test_y = np.load('/content/drive/MyDrive/data.T/testY.npy')

In [ ]:
test_dip = np.zeros((test_y.shape[0],4,3))
for i in range(test_y.shape[0]):
  for j in range(3):
    test_dip[i,j] = d[test_y[i][j].nonzero()[0][0]]

In [ ]:
for i in range(test_x.shape[0]):
    scaler = StandardScaler().fit(test_x[i])
    test_x[i] = scaler.transform(test_x[i])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.metrics import BinaryAccuracy
from keras.metrics import RootMeanSquaredError
from keras import regularizers

In [ ]:
model = Sequential()
model.add(LSTM(3, return_sequences=True, 
               kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.01),
               bias_regularizer=regularizers.L2(0.01),
               activity_regularizer=regularizers.L2(0.01)))
model.compile(optimizer='SGD', loss='mean_squared_error', metrics = RootMeanSquaredError())

In [ ]:
model.fit(x, dip_y, epochs=20, batch_size=32)

Epoch 1/20
469/469 [==============================] - 4s 3ms/step - loss: 1.8801 - root_mean_squared_error: 0.1444
Epoch 2/20
469/469 [==============================] - 1s 3ms/step - loss: 0.5154 - root_mean_squared_error: 0.0646
Epoch 3/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0450 - root_mean_squared_error: 0.0406
Epoch 4/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0190 - root_mean_squared_error: 0.0393
Epoch 5/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0163 - root_mean_squared_error: 0.0390
Epoch 6/20
469/469 [==============================] - 1s 3ms/step - loss: 0.0141 - root_mean_squared_error: 0.0388
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0123 - root_mean_squared_error: 0.0387
Epoch 8/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0108 - root_mean_squared_error: 0.0386
Epoch 9/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0095

In [ ]:
test = model.predict(test_x)

8/8 [==============================] - 0s 2ms/step


In [ ]:
m = RootMeanSquaredError()
m.update_state(test_dip[0], test[0])
m.result().numpy()*1000

40.17862305045128

In [ ]:
test[0]

array([[ 0.00192036,  0.00023212,  0.02144967],
       [ 0.00459053, -0.00068638,  0.02986995],
       [ 0.00630837, -0.0020385 ,  0.03407912],
       [ 0.00800999, -0.00343212,  0.03514498]], dtype=float32)

In [ ]:
test_dip[0]

array([[-0.040927, -0.031953,  0.081446],
       [-0.046036, -0.032946,  0.081814],
       [-0.040678, -0.029895,  0.082077],
       [ 0.      ,  0.      ,  0.      ]])

In [ ]:
err = 0
for i in range(test.shape[0]):
  for j in range(3):
    err += (((test_dip[i][j][0]-test[i][j][0])*1000)**2+((test_dip[i][j][1]-test[i][j][1])*1000)**2+((test_dip[i][j][2]-test[i][j][2])*1000)**2)**(1/2)
print(err/750)

70.47783819007859
